## Importando bibliotecas

In [1]:
import pandas as pd

## Processando dataset socioeconômico

In [10]:
df = pd.read_csv('../data/raw/socioeconomico.csv', sep=';')
df.dtypes

BAIRRO                    object
RPA                        int64
MICROREGIAO              float64
DIST_MARCO_ZERO          float64
HECTARES                 float64
HABITANTES                 int64
NUM_HOMENS                 int64
POR_HOMENS               float64
NUM_MULHERES               int64
POR_MULHERES             float64
0_4ANOS                    int64
0_4ANOS%                 float64
5_14ANOS                   int64
5_14ANOS%                float64
15_17ANOS                  int64
15_17ANOS%               float64
18_24ANOS                  int64
18_24ANOS%               float64
25_59ANOS                  int64
25_59ANOS%               float64
60_ANOS                    int64
60_ANOS%                 float64
BRANCA                   float64
PRETA                    float64
PARDA                    float64
AMARELA                  float64
INDIGENA                 float64
ALFABETIZADOS            float64
CRESCIMENTO_ANUAL        float64
DENSIDADE                float64
DOMICILIOS

## Processando dataset de calendário

In [56]:
df = pd.read_csv('../data/raw/calendario.csv', sep=';')
df = df.drop(['description', 'raw_description', 'link'], axis=1)
df.columns = ['DATA', 'NOME', 'TIPO', 'CODIGO_TIPO']
df['NOME'] = df['NOME'].apply(lambda x: x.upper())
df['TIPO'] = df['TIPO'].apply(lambda x: x.upper())
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')
print(df.dtypes)

df.to_hdf('../data/processed/calendario.h5', key='data')

DATA           datetime64[ns]
NOME                   object
TIPO                   object
CODIGO_TIPO             int64
dtype: object


## Processando dataset pluviométrico

In [76]:
df = pd.read_csv('../data/raw/pluviometrico.csv', sep=';')
df = df.drop(['Acumulado'], axis=1)
df['MES'] = df['Mês/Ano'].apply(lambda x: x.split('/')[0])
df['ANO'] = df['Mês/Ano'].apply(lambda x: x.split('/')[1])
df = df.drop(['Posto', 'Mês/Ano'], axis=1)
group = df.groupby(['ANO', 'MES']).mean().reset_index()
df = pd.melt(group, ['ANO', 'MES'], group.columns.values[2:], 'DIA', 'INDICE')

map_month = {
    'jan': '01',
    'fev': '02',
    'mar': '03',
    'abr': '04',
    'mai': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'set': '09',
    'out': '10',
    'nov': '11',
    'dez': '12'}

df['MES'] = df['MES'].apply(lambda x: map_month[x])
df['DATA'] = df[['DIA', 'MES', 'ANO']].apply(lambda x: x[0] + '/' + x[1] + '/' + x[2], axis=1)
df = df.drop(['DIA', 'MES', 'ANO'], axis=1)[['DATA', 'INDICE']]
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y', errors='coerce')
df = df.dropna()
print(df.dtypes)

df.to_hdf('../data/processed/pluviometrico.h5', key='data')

DATA      datetime64[ns]
INDICE           float64
dtype: object
